# Gradnja z modeli Mistral

## Uvod

V tej lekciji bomo obravnavali:
- Raziskovanje različnih modelov Mistral
- Razumevanje primerov uporabe in scenarijev za vsak model
- Vzorci kode prikazujejo edinstvene lastnosti posameznega modela.


## Mistral modeli

V tej lekciji bomo raziskali 3 različne Mistral modele:
**Mistral Large**, **Mistral Small** in **Mistral Nemo**.

Vsak od teh modelov je brezplačno na voljo na tržnici modelov Github. Koda v tem zvezku bo uporabljala te modele za izvajanje kode. Več podrobnosti o uporabi Github modelov za [prototipiranje z AI modeli](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) najdete tukaj.


## Mistral Large 2 (2407)
Mistral Large 2 je trenutno vodilni model podjetja Mistral, zasnovan za poslovno uporabo.

Model je nadgradnja izvirnega Mistral Large in ponuja:
- Večje kontekstno okno – 128k v primerjavi z 32k
- Boljšo zmogljivost pri matematičnih in programerskih nalogah – povprečna natančnost 76,9 % v primerjavi s 60,4 %
- Izboljšano večjezično delovanje – podprti jeziki so: angleščina, francoščina, nemščina, španščina, italijanščina, portugalščina, nizozemščina, ruščina, kitajščina, japonščina, korejščina, arabščina in hindijščina.

Zaradi teh lastnosti je Mistral Large odličen pri:
- *Retrieval Augmented Generation (RAG)* – zaradi večjega kontekstnega okna
- *Function Calling* – ta model ima vgrajeno klicanje funkcij, kar omogoča povezovanje z zunanjimi orodji in API-ji. Klice je mogoče izvajati vzporedno ali zaporedno, enega za drugim.
- *Generiranje kode* – model je izjemen pri generiranju kode v Pythonu, Javi, TypeScriptu in C++.


### Primer RAG z uporabo Mistral Large 2


V tem primeru uporabljamo Mistral Large 2 za izvajanje RAG vzorca na besedilnem dokumentu. Vprašanje je napisano v korejščini in sprašuje o avtorjevih dejavnostih pred fakulteto.

Uporablja Cohere Embeddings Model za ustvarjanje vdelav besedilnega dokumenta in vprašanja. Za ta primer uporablja Python paket faiss kot shrambo vektorjev.

Poziv, poslan modelu Mistral, vključuje tako vprašanja kot tudi pridobljene odseke, ki so podobni vprašanju. Model nato poda odgovor v naravnem jeziku.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small je še en model iz družine Mistral, ki spada v kategorijo premier/enterprise. Kot že ime pove, gre za majhen jezikovni model (SLM). Prednosti uporabe Mistral Small so:
- Prihranek stroškov v primerjavi z Mistral LLM-i, kot sta Mistral Large in NeMo – do 80 % nižja cena
- Nizka zakasnitev – hitrejši odziv v primerjavi z Mistralovimi LLM-i
- Prilagodljivost – lahko se uporablja v različnih okoljih z manj omejitvami glede potrebnih virov.

Mistral Small je odlična izbira za:
- Besedilne naloge, kot so povzemanje, analiza sentimenta in prevajanje.
- Aplikacije, kjer je veliko pogostih zahtevkov, saj je cenovno ugoden
- Naloge s kodo, kjer je pomembna nizka zakasnitev, kot so pregled in predlogi kode


## Primerjava med Mistral Small in Mistral Large

Za prikaz razlik v zakasnitvi med Mistral Small in Large zaženite spodnje celice.

Opazili boste razliko v odzivnem času med 3-5 sekundami. Bodite pozorni tudi na dolžino in slog odgovorov pri istem pozivu.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

V primerjavi z drugima dvema modeloma, omenjenima v tej lekciji, je Mistral NeMo edini brezplačen model z licenco Apache2.

Velja za nadgradnjo prejšnjega odprtokodnega LLM modela Mistral, Mistral 7B.

Nekatere druge značilnosti modela NeMo so:

- *Učinkovitejša tokenizacija:* Ta model uporablja Tekken tokenizer namesto bolj pogosto uporabljenega tiktoken. To omogoča boljšo zmogljivost pri več jezikih in pri kodi.

- *Dodatno učenje (finetuning):* Osnovni model je na voljo za dodatno učenje. To omogoča večjo prilagodljivost za primere uporabe, kjer je dodatno učenje potrebno.

- *Izvorno klicanje funkcij* - Tako kot Mistral Large je bil tudi ta model treniran za klicanje funkcij. Zaradi tega je edinstven, saj je eden prvih odprtokodnih modelov, ki to omogoča.


## Mistral NeMo

V primerjavi z drugima dvema modeloma, o katerih govorimo v tej lekciji, je Mistral NeMo edini brezplačen model z licenco Apache2.

Velja za nadgradnjo prejšnjega odprtokodnega LLM iz Mistrala, Mistral 7B.

Nekatere druge značilnosti modela NeMo so:

- *Učinkovitejša tokenizacija:* Ta model uporablja Tekken tokenizer namesto bolj pogosto uporabljenega tiktoken. To omogoča boljšo zmogljivost pri več jezikih in kodi.

- *Dodatno učenje (finetuning):* Osnovni model je na voljo za dodatno učenje. To omogoča večjo prilagodljivost za primere uporabe, kjer je dodatno učenje potrebno.

- *Izvorno klicanje funkcij* - Tako kot Mistral Large je bil tudi ta model treniran za klicanje funkcij. Zaradi tega je edinstven, saj je eden prvih odprtokodnih modelov, ki to omogoča.


### Primerjava tokenizatorjev

V tem primeru si bomo ogledali, kako Mistral NeMo obravnava tokenizacijo v primerjavi z Mistral Large.

Oba primera uporabljata enak poziv, vendar bi morali opaziti, da NeMo vrne manj žetonov kot Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Učenje se tukaj ne konča, nadaljujte svojo pot

Ko zaključite to lekcijo, si oglejte našo [zbirko za učenje generativne umetne inteligence](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) in še naprej nadgrajujte svoje znanje o generativni umetni inteligenci!



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko samodejni prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem izvirnem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
